<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#F21DL-Data-Mining-&amp;-Machine-Learning" data-toc-modified-id="F21DL-Data-Mining-&amp;-Machine-Learning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>F21DL Data Mining &amp; Machine Learning</a></span><ul class="toc-item"><li><span><a href="#Coursework-2-:--Convolutional-Network-for-Image-Classification" data-toc-modified-id="Coursework-2-:--Convolutional-Network-for-Image-Classification-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Coursework 2 :  Convolutional Network for Image Classification</a></span><ul class="toc-item"><li><span><a href="#Imports-&amp;-constants" data-toc-modified-id="Imports-&amp;-constants-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Imports &amp; constants</a></span></li></ul></li></ul></li></ul></div>

# F21DL Data Mining & Machine Learning

## Coursework 2 :  Convolutional Network for Image Classification
---
*ACCAD Dimitri, AUZIMOUR Antoine, DELTEL Clarence, DI MARTINO Thomas*

This notebook presents our work in the research question. 
In our research work, we wanted to benchmark the performances of a more common Deep Learning algorithm used to deal with data that have an architecture in it, whether it is spatial or temporal: the convolutional neural networks.

<figure style="margin-top, margin-bottom:10px; text-align: center">
        <img src="https://www.learnopencv.com/wp-content/uploads/2017/11/convolution-example-matrix.gif">
    <figcaption style="display:block;margin-left:auto;margin-right:auto"><u>Animation of the convolutional process (source: <a href="https://www.learnopencv.com/image-classification-using-convolutional-neural-networks-in-keras/">link</a>)</u></figcaption>
</figure>

These neural networks use the process of convolution (c.f. above) to generate <b>features maps</b> that are dense in information and that can help a classifier to have more ease to find the correct class.

This process of extracting features from images is transferable to a lot of different tasks: given a dataset where we want to discriminate cats from dogs, cars from motorcycles, a CNN will always look for patterns, more or less complex, depending on the depth of the network, that makeup any object (it could be a line, a curve, a round, gaps in colors...). This is the combination of these objects that leads a CNN to detect a given object in an image.

In this notebook, we do not use Transfer Learning, we just developed a simple CNN with its convolutional and fully-connected layers.

Now for the code part !

### Imports & constants

In [1]:
# MAIN IMPORTS

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import numpy as np
import sklearn.preprocessing
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import PIL
import tensorflow as tf

# KERAS IMPORTS

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Conv2D, Dropout, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras_tqdm import TQDMNotebookCallback

# SKLEARN IMPORTS

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

NUM_FOLDS = 10
ORIGINAL_IMAGE_SHAPE = (48, 48)
VGG_IMAGE_SHAPE = (224,224)
NUM_CLASSES = 10
LR = 0.001
BATCH_SIZE = 4
SEED = 42
IMG_FOLDER = "data"
ROOT_FOLDER = os.getcwd()
NB_EPOCHS = 20

Using TensorFlow backend.


To make it easier for Keras to work with the images, we will save all of them as real .png file in a data folder.

In [2]:
def from_csv_to_disk(PATH_TO_CSV="x_train_gr_smpl.csv", PATH_TO_DISK="data"):
    print("Opening csv..")
    data = pd.read_csv(PATH_TO_CSV)
    print("Csv opened..")
    print("Saving images to {}".format(os.path.join(os.getcwd(),PATH_TO_DISK)))
    for index, row in tqdm(data.iterrows()):
        np_row = row.to_numpy()
        img = np_row.reshape(ORIGINAL_IMAGE_SHAPE).astype(np.float32)
        cv2_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2.imwrite(
            os.path.join(os.path.join(os.getcwd(),PATH_TO_DISK), f"img{index}.png"),#path to img
            cv2_img
        )

In [3]:
#from_csv_to_disk()

We now define the useful classes for our problem:
 - The model class that we will use to interact with our model
 - The Generator Keras object that will provide data to our network

In [27]:
def get_generator(dataframe, path_column_name="imgpath", class_column_name="class", is_test = False):
    """
    Dataframe should contain two columns:
     - img path column (default name: 'imgpath')
     - img class column (default name: 'class')
     
    If is_test: return a single generator
    If is_test is False: return a tuple of train and val datasets
    """
    
    
    if (is_test):
        
        generator = ImageDataGenerator(rescale=1./255)
        
        test_generator = generator.flow_from_dataframe(
            directory=os.path.join(ROOT_FOLDER, IMG_FOLDER),
            shuffle=False,
            dataframe=dataframe,  
            x_col=path_column_name, 
            y_col=class_column_name, 
            class_mode="categorical", 
            target_size=ORIGINAL_IMAGE_SHAPE,
            color_mode="rgb", #the last two parameters induce the resizing to (224,224,3)
            seed=SEED
        )
        
        return test_generator
    
    else:      
    
        generator = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2, # set validation split
            #data augmentation
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )
        train_generator = generator.flow_from_dataframe(
            directory=os.path.join(ROOT_FOLDER, IMG_FOLDER),
            dataframe=dataframe,  
            x_col=path_column_name, 
            y_col=class_column_name, 
            class_mode="categorical", 
            target_size=ORIGINAL_IMAGE_SHAPE,
            color_mode="rgb", #the last two parameters induce the resizing to (224,224,3)
            batch_size=BATCH_SIZE,
            seed=SEED,
            subset='training') 

        validation_generator = generator.flow_from_dataframe(
            directory=os.path.join(ROOT_FOLDER, IMG_FOLDER),
            dataframe=dataframe,  
            x_col=path_column_name, 
            y_col=class_column_name, 
            class_mode="categorical", 
            target_size=ORIGINAL_IMAGE_SHAPE,
            color_mode="rgb", #the last two parameters induce the resizing to (224,224,3)
            batch_size=BATCH_SIZE,
            seed=SEED,
            subset='validation') 
    
        return train_generator, validation_generator

In [28]:
class CNN:
    
    def __init__(self):
        
        
        x = Input(shape = ORIGINAL_IMAGE_SHAPE + (3,))
        
        y = Conv2D(16,kernel_size=(3,3), activation='relu')(x)
        y = BatchNormalization()(y)
        
        y = Conv2D(32,kernel_size=(3,3), activation='relu')(y)
        y = BatchNormalization()(y)
        y = MaxPooling2D(pool_size=(2, 2))(y)
        y = Dropout(0.2)(y)
        
        y = Conv2D(32,kernel_size=(3,3), activation='relu')(y)
        y = BatchNormalization()(y)
        
        y = Conv2D(16,kernel_size=(3,3), activation='relu')(y)
        y = MaxPooling2D(pool_size=(2, 2))(y)
        y = Dropout(0.2)(y)
        
        y = Flatten()(y)
        
        y = Dense(1024, activation="relu")(y)
        y = Dropout(0.3)(y)
        y = Dense(256, activation="relu")(y)
        y = Dropout(0.3)(y)
        y = Dense(64, activation="relu")(y)
        y = Dropout(0.3)(y)
        y = Dense(NUM_CLASSES, activation="softmax")(y)
        self.model = Model(x, y)
        
        opt = Adam(learning_rate=LR)
        
        self.model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
        
    def train(self, datagen):
        """
        Make sure to create the datagen object with the 'get_generator' method: it should be a tuple of training and validation generators
        """
        train_gen, val_gen = datagen
        
        filepath="cnn-{epoch:02d}-{val_accuracy:.2f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
        
        return self.model.fit_generator(    
            train_gen,
            epochs=NB_EPOCHS,
            steps_per_epoch = train_gen.samples,
            validation_data = val_gen, 
            validation_steps = val_gen.samples,
            verbose=0, 
            callbacks=[
                TQDMNotebookCallback(), 
                checkpoint,
                TensorBoard(
                    log_dir='.\\logs', histogram_freq=1, batch_size=BATCH_SIZE, write_graph=True, write_grads=True, write_images=True, update_freq='epoch'
                )
            ]
        )
    
    def test(self, datagen, steps=1):
        """
        Make sure to create the datagen object with the 'get_generator' method
        """
        return self.model.evaluate_generator(datagen, steps=steps)
    
    def predict(self, datagen):
        """
        Make sure to create the datagen object with the 'get_generator' method        
        """
        return self.model.predict_generator(datagen)
    
    def load_from_fileweights(self, path):
        """
        
        """
        self.model.load_weights(os.path.join(os.getcwd(), path))

We will now load data and launching training sessions using the same 10-fold validation procedure.

In [29]:
images_path = np.array(
                sorted(os.listdir(os.path.join(ROOT_FOLDER, IMG_FOLDER)), 
                     key=lambda x: eval(x[3:][:-4])# we sort images with their index number
                    )
                )

In [30]:
classes = pd.read_csv("y_train_smpl.csv").to_numpy()

In [31]:
c = pd.DataFrame(classes)
c = c.astype(str)
df = pd.concat([pd.DataFrame(np.expand_dims(images_path, axis=1)), c], axis=1)
df.columns = ["imgpath", "class"]
df.head()

,imgpath,class
0,img0.png,0
1,img1.png,0
2,img2.png,0
3,img3.png,0
4,img4.png,0


Our dataframe has now the correct information, we can create the generator.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(df["imgpath"], df["class"], test_size=0.20, random_state=SEED)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

model = CNN()

datagen_train = get_generator(train_df) # tuple (train_gen, val_ge)
datagen_test = get_generator(test_df, is_test=True) # single generator for test

model.load_from_fileweights("cnn-08-0.99.hdf5")

#hist = model.train(datagen_train)

Found 8103 validated image filenames belonging to 10 classes.
Found 2025 validated image filenames belonging to 10 classes.
Found 2532 validated image filenames belonging to 10 classes.


In [33]:
print("\t\tTested metrics are:", model.model.metrics_names)
print("\t\tPerf: ",model.test(datagen_test))

		Tested metrics are: ['loss', 'accuracy']
		Perf:  [0.00017992999346461147, 1.0]


In [34]:
y_pred = model.predict(datagen_test)

Test performances of our CNN

In [38]:
pd.DataFrame(metrics.classification_report(y_test,np.argmax(y_pred, axis=1).astype(str) ,  output_dict=True))

,0,1,2,3,4,5,6,7,8,9,accuracy,macro avg,weighted avg
precision,0.996337,0.964912,1.0,1.000000,1.000000,1.0,1.0,1.0,0.980926,0.927536,0.989336,0.986971,0.989597
recall,0.947735,0.997409,1.0,0.992424,0.995134,1.0,1.0,1.0,0.986301,0.969697,0.989336,0.988870,0.989336
f1-score,0.971429,0.980892,1.0,0.996198,0.997561,1.0,1.0,1.0,0.983607,0.948148,0.989336,0.987783,0.989341
support,287.000000,386.000000,87.0,264.000000,411.000000,465.0,157.0,44.0,365.000000,66.000000,0.989336,2532.000000,2532.000000


In [46]:
del train_df
del test_df
del model
from keras import backend as K
import gc
K.clear_session()
gc.collect()

86869